In [1]:
import requests
import json
import pandas as pd
from datetime import datetime
import statistics
from IPython.display import HTML, display
from wimbledon.vis.Visualise import DataHandlers

In [2]:
with open('github.token', 'r') as f:
    token = f.read().strip()

headers = {"Authorization": "Bearer " + token}


def run_query(query):  # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post('https://api.github.com/graphql', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))


GitHub emojis
=======

Project issues that members of the REG team have selected, "LAUGH" are the ones they would most like to be assigned to, "THUMBS_UP" they would also be happy with, "THUMBS_DOWN" they would not be happy with.

In [3]:
emojis = {'CONFUSED': '😕',
          'EYES': '👀',
          'HEART': '❤️',
          'HOORAY': '🎉',    
          'ROCKET': '🚀',
          'THUMBS_DOWN': '❌',
          'THUMBS_UP': '👍',
          'LAUGH': '✅'}

In [4]:
fc = DataHandlers.Forecast()  # get data from forecast

Enpoint: whoami | Time: 0.5002057310000001 | Cache: False
Enpoint: people/460554 | Time: 0.5038793319999999 | Cache: False

AUTHENTICATED USER:
Jack Roberts jroberts@turing.ac.uk

CLIENTS
Enpoint: clients | Time: 0.466326145 | Cache: False
PROJECTS
Enpoint: projects | Time: 0.6085192239999997 | Cache: False
ROLES
Enpoint: roles | Time: 0.570470979 | Cache: False
PEOPLE
Enpoint: people | Time: 0.6045541830000003 | Cache: False
PLACEHOLDERS
Enpoint: placeholders | Time: 0.48135447900000017 | Cache: False
MILESTONES
Enpoint: milestones | Time: 0.4594184769999998 | Cache: False
ASSIGNMENTS
Enpoint: assignments | Time: 0.7952631750000005 | Cache: False
DONE! (5.0s)


REG team
=====

We need a list of the team members full names from Forecast to compare with their names on GitHub. They match most of the time except when there is no GitHub name. Mapping below covers the rest of the team. We need to update this list if someone new is added to Forecast who doesn't have a matching GitHub name.

In [5]:
names = list(fc.people.full_name)
names.remove('Giovanni Colavizza')
names.remove('Miguel Morin')
names.remove('Mohammed Ali Al-Badri')
names

['Oliver Strickson',
 'Angus Williams',
 'David Beavan',
 'Evelina Gabasova',
 'James Geddes',
 'James Hetherington',
 'Louise Bowler',
 "Martin O'Reilly",
 'May Yong',
 'Nick Barlow',
 'Radka Jersakova',
 'Timothy Hobson',
 'Tomas Lazauskas',
 'Camila Rangel Smith',
 'Sarah Gibson',
 'James Robinson',
 'Eric Daub',
 'Jim Madge',
 'Amber Raza',
 'Kasra Hosseini',
 'Oscar Giles',
 'Joel Dearden',
 'Amaani Hoddoon',
 'Jack Roberts',
 'Oliver Forrest',
 'Gabriel Hanganu',
 'Ed Chalstrey',
 'Ashwini Venkatasubramaniam',
 'Roly Perera',
 'Federico Nanni',
 'Anna Hadjitofi',
 'Flora Roumpani']

Find a team member's availability for a date range
---

In [6]:
def get_person_availability(peopledf, person, start_date, end_date):
    """Get the mean of a person's FTE proportion available for the start to end datetime objects"""
    if isinstance(person, str):
        try:
            person = fc.get_person_id(person)
        except:
            return 0.0
    peopledf = peopledf[(peopledf.index >= start_date) & (peopledf.index <= end_date)]
    try:
        availability_range = peopledf[person]
    except:
        return 0.0
    average_availability = statistics.mean(availability_range)
    return round(average_availability, 2)

In [7]:
peopledf = 1 - fc.people_totals.resample('MS').mean()  # pandas df for team members availability
start_date = datetime(2019, 10, 1)
end_date = datetime(2020, 10, 1)
print(get_person_availability(peopledf, "Ed Chalstrey", start_date, end_date))
print(get_person_availability(peopledf, 458589, start_date, end_date))

0.65
1.0


Projects should have a consistent resource required number over the months they aren't 0, but may be useful to look at the resource requirement over a specific date range:

In [8]:
def get_project_requirement(projectdf, project, start_date, end_date):
    """Get the mean of a project's FTE requirement for the start to end datetime objects"""
    if isinstance(project, str):
        try:
            project = fc.get_project_id(project)
        except:
            return 0.0
    projectdf = projectdf[(projectdf.index >= start_date) & (projectdf.index <= end_date)]
    try:
        requirement_range = projectdf[project]
    except:
        return 0.0
    average_requirement = statistics.mean(requirement_range)
    return round(average_requirement, 2)

In [9]:
projectdf = fc.project_resourcereq.resample('MS').mean()  # pandas df for team members availability
print(get_project_requirement(projectdf, "CHANCE", start_date, end_date))
print(get_project_requirement(projectdf, 2061048, start_date, end_date))

0.62
0.23


Create queries with GraphQL to get the emojis for each GitHub issue
===

https://developer.github.com/v4/guides/forming-calls/

In [10]:
query = """
{
  repository(owner:"alan-turing-institute", name:"Hut23") {
    issue(number:X) {
          number
          title
          url

          reactionGroups {
            content
            users(first:20) {
                edges {
                    node {
                        login
                        name
                    }
                }
            }
            }
    }
  }
}
"""

Check if a GitHub issue has emojis, based on the result of GraphQL query
----

In [11]:
def query_result_contains_emojis(query_result):
    for reaction in query_result['data']['repository']['issue']['reactionGroups']:
        if len(reaction['users']['edges']) > 0:
            return True
    return False

In [12]:
def get_preference_data(fc):
    """Get each team members preference emoji for all projects with a GitHub issue"""
    issues = fc.projects["GitHub"].dropna()  # Get list of GitHub issues for projects
    gid_mapping = {  # People without their full names on github.
     'myyong': 'May Yong',
     'nbarlowATI': 'Nick Barlow',
     'thobson88': 'Timothy Hobson',
     'miguelmorin': 'Miguel Morin',
     'OscartGiles': 'Oscar Giles',
     'AshwiniKV': 'Ashwini Venkatasubramaniam',
    }
    emoji_mapping = {'CONFUSED': '😕',
          'EYES': '👀',
          'HEART': '❤️',
          'HOORAY': '🎉',    
          'ROCKET': '🚀',
          'THUMBS_DOWN': '❌',
          'THUMBS_UP': '👍',
          'LAUGH': '✅'}
    names = list(fc.people.full_name)
    names.remove('Giovanni Colavizza')
    names.remove('Miguel Morin')
    names.remove('Mohammed Ali Al-Badri')
    names.remove('Angus Williams')
    preference_data = {
        "Person": names
    }
    for issue_num, project_id in zip(issues, issues.index):
        modified_query = query.replace("X", str(issue_num))
        result = run_query(modified_query)  # Execute the query
#         if query_result_contains_emojis(result)  # Could do this if we only want issues with emojis
        emojis = []
        for name in names: 
            emoji_name = None
            for reaction in result['data']['repository']['issue']['reactionGroups']:
                for edge in reaction['users']['edges']:
                    if edge['node']['name'] == name:
                        emoji_name = reaction['content']
                    if not emoji_name:
                        if edge['node']['login'] in gid_mapping:
                            if gid_mapping[edge['node']['login']] == name:
                                emoji_name = reaction['content']
            if emoji_name:
                emoji = emoji_mapping[emoji_name]
            else:
                emoji = "❓"
            emojis.append(emoji)
        preference_data[fc.get_project_name(project_id)] = emojis
    return preference_data

In [13]:
preference_data = get_preference_data(fc)

In [14]:
preference_data_df = pd.DataFrame(preference_data).set_index('Person')
preference_data_df

,Sargasso (Intel Sparse),DetectorChecker,HSBC EDS,AI for City Planning,Reproducible Research,PDQ,Crypto Federated Learning (SHEEP),The Turing Way,Fitbit classification,Parallel Monte Carlo,...,Interpretable models,Census 2021 backbrief,Solar Panel Detection,NoCell - Phase 2,Hate Speech,SAIL multimorbidity,QUIPP - Synthetic data,WAYS WP1 - Extending BackFillz,WAYS WP2 - Auto-generating precursor visualisations,Survival outcome - Roche
Person,,,,,,,,,,,,,,,,,,,,,
Oliver Strickson,❓,❓,❓,❓,❓,❓,❓,❌,❓,❓,...,❓,👍,👍,✅,❌,👍,👍,👍,👍,❓
David Beavan,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓,...,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓
Evelina Gabasova,❓,❓,❓,❓,❓,❓,❓,✅,👍,👍,...,❓,❓,👍,👍,👍,👍,👍,👍,👍,❓
James Geddes,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓,...,❓,👍,❓,✅,❓,❓,❓,❓,❓,❓
James Hetherington,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓,...,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓
Louise Bowler,❓,❓,❓,❓,❓,❓,❓,✅,❌,❌,...,❓,👍,👍,❌,👍,👍,👍,👍,👍,❓
Martin O'Reilly,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓,...,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓
May Yong,❓,❓,❓,❓,❓,❓,❓,❌,❓,❓,...,❓,❌,❓,❌,👍,❓,❌,👍,👍,❓
Nick Barlow,❓,❓,❓,❓,❓,❓,❓,❌,👍,❌,...,❓,❌,👍,👍,👍,👍,👍,👍,👍,❓


In [15]:
preference_data_df["CHANCE"]["Ed Chalstrey"]

'✅'

Get the projects for which resources (team members) are required and display availability and preference for each team member who has with reacted with an emoji on the GitHub issue
----

In [16]:
fc.project_resourcereq.resample('MS').mean()

id,1684536,1684539,1723251,1723252,1723253,1723254,1723255,1723256,1723257,1723258,...,2197563,2197564,2197567,2197568,2197569,2198831,2200463,2200468,2202653,2202670
2016-11-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
2016-12-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
2017-01-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
2017-02-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
2017-03-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
2017-04-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
2017-05-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
2017-06-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
2017-07-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
2017-08-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0


In [17]:
def get_preferences(fc, preference_data_df, first_date=False, last_date=False, person=False, project=False, positive_only=False, emojis_only=False):
    resreqdf = fc.project_resourcereq.resample('MS').mean() # grouped by month and mean taken
    if person:
        names = [person]
    else:
        names = list(preference_data_df.index)
    data = {
        "Person": names
    }
    issues = fc.projects["GitHub"].dropna().values
    peopledf = 1 - fc.people_totals.resample('MS').mean()
    projectdf = fc.project_resourcereq.resample('MS').mean()  # pandas df for team members availability
    if project:
        if isinstance(project, str):
            try:
                project = fc.get_project_id(project)
            except:
                pass
    for project_id in resreqdf:  # some of these have no GitHub issue
        if not project or project == project_id:
            date_indices = resreqdf.index[resreqdf[project_id] > 0]
            if len(date_indices) > 0:  # if at least one month in the dataframe has a resource requirement of more than 0 FTE
                issue_num = fc.projects.loc[project_id, "GitHub"]
                if issue_num in issues:  # if this project has a GitHub issue
    #                 start_date = fc.projects.loc[project_id, "start_date"]
    #                 end_date = fc.projects.loc[project_id, "end_date"]
                    first_resreq_date = date_indices[0].strftime("%Y-%m-%d")
                    last_resreq_date = date_indices[-1].strftime("%Y-%m-%d")
                    resreq = get_project_requirement(projectdf, project_id, first_resreq_date, last_resreq_date)
                    project_title = fc.projects.loc[project_id, "name"]
                    emoji_data = []
                    for name in names:
#                         print(name)
                        person_availability = get_person_availability(peopledf, name, first_resreq_date, last_resreq_date)
                        percentage_availability = round((person_availability / resreq) * 100, 2)
                        emoji = preference_data_df[project_title][name]
                        if (not person and not project) or not positive_only or emoji == '✅' or emoji == '👍':
                            if emojis_only:
                                emoji_data.append(emoji)
                            else:
                                emoji_data.append(emoji + " " + str(percentage_availability) + "% (" + str(person_availability) + " / " + str(round(resreq, 2)) + ")")
#                         if project and positive_only and (emoji == '❌' or emoji == '❓'):
#                             print(name, emoji)
#                             data["Person"].remove(name)
#                         else:
#                             print(name, emoji)
                            
                    if (not person and not project) or not positive_only or len(emoji_data) > 0:
                        data[project_title] = emoji_data
                        
#     print(data)
    return pd.DataFrame(data).set_index('Person')

All projects with team members preferences and their average availability between the start of project resource requirement and project end date
----

In [26]:
emoji_table = get_preferences(fc, preference_data_df, emojis_only=True).to_html()
with open('tmp.html', 'w') as f:
    f.write(emoji_table)
    
print(emoji_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Living With Machines</th>
      <th>Probabilistic FEM</th>
      <th>Safe Haven</th>
      <th>Learning Machines</th>
      <th>UQM^3</th>
      <th>Auto DL</th>
      <th>All in One Cancer imaging optimisation</th>
      <th>Uncertainty in Government Modelling</th>
      <th>Interpretable AI for health</th>
      <th>Micro-location analytics</th>
      <th>Machine learning for cancer</th>
      <th>Urban systems resilience</th>
      <th>Visualisation: Roy Ruddle TF project</th>
      <th>Logics for DS</th>
      <th>DFTG Commons scoping</th>
      <th>DFTG Commons WP 3.4/3.5</th>
      <th>CHANCE</th>
      <th>Scaleable Monte Carlo</th>
      <th>Wrattler 2019-2020</th>
      <th>Census 2021 backbrief</th>
      <th>Solar Panel Detection</th>
      <th>NoCell - Phase 2</th>
      <th>SAIL multimorbidity</th>
    </tr>
    <tr>
      <th>Person</th>
      <th></th>
      <th><

<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Living With Machines</th>\n      <th>AIDA-Lloyds</th>\n      <th>Probabilistic FEM</th>\n      <th>Safe Haven</th>\n      <th>Learning Machines</th>\n      <th>UQM^3</th>\n      <th>Auto DL</th>\n      <th>All in One Cancer imaging optimisation</th>\n      <th>Fusion modelling- IDEA</th>\n      <th>Uncertainty in Government Modelling</th>\n      <th>Interpretable AI for health</th>\n      <th>Micro-location analytics</th>\n      <th>Machine learning for cancer</th>\n      <th>Urban systems resilience</th>\n      <th>NCSC</th>\n      <th>Visualisation: Roy Ruddle TF project</th>\n      <th>SPF TPS 2019+</th>\n      <th>Logics for DS</th>\n      <th>Medevac Decision Support</th>\n      <th>NHS Medication Safety</th>\n      <th>UCLH Cancer Waiting Times</th>\n      <th>NHS Scotland Cancer Waiting Times</th>\n      <th>DFTG Commons scoping</th>\n      <th>DFTG Commons WP 3.4/3.5</th>\n      <th>CHANCE</th>\n      <th>Scaleable Monte Carlo</th>\n      <th>Wrattler 2019-2020</th>\n      <th>Census 2021 backbrief</th>\n      <th>Solar Panel Detection</th>\n      <th>NoCell - Phase 2</th>\n      <th>The Turing Way - Phase 2</th>\n      <th>Hate Speech</th>\n      <th>SAIL multimorbidity</th>\n    </tr>\n    <tr>\n      <th>Person</th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Oliver Strickson</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Angus Williams</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>David Beavan</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Evelina Gabasova</th>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>James Geddes</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>✅</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>James Hetherington</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>✅</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Louise Bowler</th>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>✅</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>Martin O\'Reilly</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>May Yong</th>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>✅</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>✅</td>\n      <td>❌</td>\n      <td>✅</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Nick Barlow</th>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>✅</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>✅</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>✅</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>Radka Jersakova</th>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>Timothy Hobson</th>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>Tomas Lazauskas</th>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>✅</td>\n      <td>✅</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>Camila Rangel Smith</th>\n      <td>👍</td>\n      <td>✅</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>✅</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>✅</td>\n      <td>✅</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>✅</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>Sarah Gibson</th>\n      <td>✅</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>✅</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>James Robinson</th>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>✅</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>✅</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>Eric Daub</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n    </tr>\n    <tr>\n      <th>Jim Madge</th>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n    </tr>\n    <tr>\n      <th>Amber Raza</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Kasra Hosseini</th>\n      <td>✅</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>Oscar Giles</th>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>✅</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>✅</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>✅</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>Joel Dearden</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Amaani Hoddoon</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Jack Roberts</th>\n      <td>✅</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>✅</td>\n      <td>✅</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>Oliver Forrest</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Gabriel Hanganu</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Ed Chalstrey</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>✅</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>✅</td>\n      <td>✅</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>✅</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>Ashwini Venkatasubramaniam</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>✅</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>✅</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>Roly Perera</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n    </tr>\n  </tbody>\n</table>\n<p>29 rows × 33 columns</p>

In [19]:
get_preferences(fc, preference_data_df)

,Living With Machines,Probabilistic FEM,Safe Haven,Learning Machines,UQM^3,Auto DL,All in One Cancer imaging optimisation,Uncertainty in Government Modelling,Interpretable AI for health,Micro-location analytics,...,Logics for DS,DFTG Commons scoping,DFTG Commons WP 3.4/3.5,CHANCE,Scaleable Monte Carlo,Wrattler 2019-2020,Census 2021 backbrief,Solar Panel Detection,NoCell - Phase 2,SAIL multimorbidity
Person,,,,,,,,,,,,,,,,,,,,,
Oliver Strickson,❓ 102.78% (0.74 / 0.72),❓ 100.0% (0.5 / 0.5),❓ 27.27% (0.36 / 1.32),❓ 18.92% (0.35 / 1.85),👍 64.0% (0.32 / 0.5),❌ 100.0% (0.5 / 0.5),👍 71.0% (0.71 / 1.0),❓ 100.0% (0.5 / 0.5),❌ 0.0% (0.0 / 1.0),👍 50.0% (0.5 / 1.0),...,❌ 180.0% (0.45 / 0.25),❌ 50.0% (0.5 / 1.0),❌ 13.5% (0.54 / 4.0),👍 38.0% (0.38 / 1.0),👍 100.0% (1.0 / 1.0),❓ 50.0% (0.5 / 1.0),👍 178.57% (0.5 / 0.28),👍 100.0% (0.5 / 0.5),✅ 16.0% (0.08 / 0.5),👍 200.0% (1.0 / 0.5)
David Beavan,❓ 65.28% (0.47 / 0.72),❓ 32.0% (0.16 / 0.5),❓ 6.82% (0.09 / 1.32),❓ 17.3% (0.32 / 1.85),❓ 46.0% (0.23 / 0.5),❓ 0.0% (0.0 / 0.5),❓ 47.0% (0.47 / 1.0),❓ 26.0% (0.13 / 0.5),❓ 40.0% (0.4 / 1.0),❓ 0.0% (0.0 / 1.0),...,❓ 108.0% (0.27 / 0.25),❓ 0.0% (0.0 / 1.0),❓ 11.5% (0.46 / 4.0),❓ 25.0% (0.25 / 1.0),❓ 50.0% (0.5 / 1.0),❓ 40.0% (0.4 / 1.0),❓ 0.0% (0.0 / 0.28),❓ 40.0% (0.2 / 0.5),❓ 84.0% (0.42 / 0.5),❓ 100.0% (0.5 / 0.5)
Evelina Gabasova,👍 69.44% (0.5 / 0.72),❌ 40.0% (0.2 / 0.5),❓ 8.33% (0.11 / 1.32),❓ 20.54% (0.38 / 1.85),❌ 58.0% (0.29 / 0.5),👍 0.0% (0.0 / 0.5),👍 50.0% (0.5 / 1.0),👍 34.0% (0.17 / 0.5),👍 50.0% (0.5 / 1.0),👍 0.0% (0.0 / 1.0),...,👍 128.0% (0.32 / 0.25),❌ 0.0% (0.0 / 1.0),❌ 12.5% (0.5 / 4.0),👍 31.0% (0.31 / 1.0),❌ 50.0% (0.5 / 1.0),❓ 50.0% (0.5 / 1.0),❓ 0.0% (0.0 / 0.28),👍 50.0% (0.25 / 0.5),👍 100.0% (0.5 / 0.5),👍 100.0% (0.5 / 0.5)
James Geddes,❓ 54.17% (0.39 / 0.72),❓ 40.0% (0.2 / 0.5),❓ 15.91% (0.21 / 1.32),❓ 8.11% (0.15 / 1.85),❓ 36.0% (0.18 / 0.5),❓ 34.0% (0.17 / 0.5),❓ 38.0% (0.38 / 1.0),❓ 56.0% (0.28 / 0.5),❓ 0.0% (0.0 / 1.0),❓ 38.0% (0.38 / 1.0),...,❓ 96.0% (0.24 / 0.25),❓ 0.0% (0.0 / 1.0),❓ 6.25% (0.25 / 4.0),❓ 19.0% (0.19 / 1.0),👍 50.0% (0.5 / 1.0),❓ 50.0% (0.5 / 1.0),👍 0.0% (0.0 / 0.28),❓ 50.0% (0.25 / 0.5),✅ 0.0% (0.0 / 0.5),❓ 100.0% (0.5 / 0.5)
James Hetherington,❓ 12.5% (0.09 / 0.72),✅ 0.0% (0.0 / 0.5),❓ 0.0% (0.0 / 1.32),❓ 2.7% (0.05 / 1.85),❓ 8.0% (0.04 / 0.5),❓ 0.0% (0.0 / 0.5),❓ 9.0% (0.09 / 1.0),❓ 0.0% (0.0 / 0.5),❓ 10.0% (0.1 / 1.0),❓ 0.0% (0.0 / 1.0),...,❓ 20.0% (0.05 / 0.25),❓ 0.0% (0.0 / 1.0),❓ 2.5% (0.1 / 4.0),❓ 2.0% (0.02 / 1.0),❓ 10.0% (0.1 / 1.0),❓ 0.0% (0.0 / 1.0),❓ 0.0% (0.0 / 0.28),❓ 0.0% (0.0 / 0.5),❓ 20.0% (0.1 / 0.5),❓ 20.0% (0.1 / 0.5)
Louise Bowler,👍 118.06% (0.85 / 0.72),❌ 100.0% (0.5 / 0.5),❓ 15.91% (0.21 / 1.32),❓ 31.35% (0.58 / 1.85),👍 78.0% (0.39 / 0.5),👍 50.0% (0.25 / 0.5),👍 83.0% (0.83 / 1.0),❌ 66.0% (0.33 / 0.5),👍 50.0% (0.5 / 1.0),👍 0.0% (0.0 / 1.0),...,❌ 200.0% (0.5 / 0.25),❌ 38.0% (0.38 / 1.0),❌ 19.75% (0.79 / 4.0),👍 50.0% (0.5 / 1.0),👍 100.0% (1.0 / 1.0),❓ 50.0% (0.5 / 1.0),👍 0.0% (0.0 / 0.28),👍 100.0% (0.5 / 0.5),❌ 116.0% (0.58 / 0.5),👍 200.0% (1.0 / 0.5)
Martin O'Reilly,❓ 30.56% (0.22 / 0.72),❓ 0.0% (0.0 / 0.5),❓ 0.0% (0.0 / 1.32),❓ 0.0% (0.0 / 1.85),❓ 0.0% (0.0 / 0.5),❓ 0.0% (0.0 / 0.5),❓ 19.0% (0.19 / 1.0),❓ 0.0% (0.0 / 0.5),❓ 0.0% (0.0 / 1.0),❓ 0.0% (0.0 / 1.0),...,❓ 0.0% (0.0 / 0.25),❓ 0.0% (0.0 / 1.0),❓ 0.0% (0.0 / 4.0),❓ 0.0% (0.0 / 1.0),❓ 25.0% (0.25 / 1.0),❓ 0.0% (0.0 / 1.0),❓ 0.0% (0.0 / 0.28),❓ 0.0% (0.0 / 0.5),❓ 0.0% (0.0 / 0.5),❓ 50.0% (0.25 / 0.5)
May Yong,❌ 130.56% (0.94 / 0.72),❌ 80.0% (0.4 / 0.5),❓ 13.64% (0.18 / 1.32),❓ 39.46% (0.73 / 1.85),❌ 108.0% (0.54 / 0.5),❌ 34.0% (0.17 / 0.5),✅ 94.0% (0.94 / 1.0),👍 56.0% (0.28 / 0.5),✅ 100.0% (1.0 / 1.0),❌ 0.0% (0.0 / 1.0),...,❌ 244.0% (0.61 / 0.25),❌ 25.0% (0.25 / 1.0),❌ 25.0% (1.0 / 4.0),❌ 56.0% (0.56 / 1.0),❌ 100.0% (1.0 / 1.0),❓ 50.0% (0.5 / 1.0),❌ 0.0% (0.0 / 0.28),❓ 84.0% (0.42 / 0.5),❌ 200.0% (1.0 / 0.5),❓ 200.0% (1.0 / 0.5)
Nick Barlow,👍 130.56% (0.94 / 0.72),👍 40.0% (0.2 / 0.5),❓ 8.33% (0.11 / 1.32),❓ 35.14% (0.65 / 1.85),✅ 92.0% (

In [20]:
get_preferences(fc, preference_data_df, person="Ed Chalstrey", positive_only=True)

,All in One Cancer imaging optimisation,Interpretable AI for health,Micro-location analytics,Machine learning for cancer,DFTG Commons scoping,CHANCE,Scaleable Monte Carlo,Census 2021 backbrief,Solar Panel Detection,SAIL multimorbidity
Person,,,,,,,,,,
Ed Chalstrey,✅ 29.0% (0.29 / 1.0),👍 100.0% (1.0 / 1.0),✅ 12.0% (0.12 / 1.0),✅ 79.0% (0.79 / 1.0),👍 50.0% (0.5 / 1.0),✅ 81.0% (0.81 / 1.0),👍 0.0% (0.0 / 1.0),👍 178.57% (0.5 / 0.28),✅ 150.0% (0.75 / 0.5),👍 0.0% (0.0 / 0.5)


In [21]:
get_preferences(fc, preference_data_df, project="CHANCE")

,CHANCE
Person,
Oliver Strickson,👍 38.0% (0.38 / 1.0)
David Beavan,❓ 25.0% (0.25 / 1.0)
Evelina Gabasova,👍 31.0% (0.31 / 1.0)
James Geddes,❓ 19.0% (0.19 / 1.0)
James Hetherington,❓ 2.0% (0.02 / 1.0)
Louise Bowler,👍 50.0% (0.5 / 1.0)
Martin O'Reilly,❓ 0.0% (0.0 / 1.0)
May Yong,❌ 56.0% (0.56 / 1.0)
Nick Barlow,👍 44.0% (0.44 / 1.0)


In [22]:
get_preferences(fc, preference_data_df, project="CHANCE", positive_only=True)

ValueError: arrays must all be same length